# Модель сегментации контуров людей

[датасет](https://www.kaggle.com/datasets/tapakah68/supervisely-filtered-segmentation-person-dataset)


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from model import HumanSegmentationModel

In [14]:
class HumanSegmentationDataset(Dataset):
    def __init__(self, data_dir, img_size=(256, 256)):
        self.data_dir = data_dir
        self.img_size = img_size
        self.images_dir = os.path.join(data_dir, 'images')
        self.masks_dir = os.path.join(data_dir, 'masks')
        
        self.image_files = [
            f for f in os.listdir(self.images_dir) 
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))
        ]
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_name)
        
        mask_name = os.path.splitext(img_name)[0] + '.png'
        mask_path = os.path.join(self.masks_dir, mask_name)
        
        if not os.path.exists(mask_path):
            possible_masks = [
                f for f in os.listdir(self.masks_dir) 
                if os.path.splitext(f)[0] == os.path.splitext(img_name)[0]
            ]
            if possible_masks:
                mask_path = os.path.join(self.masks_dir, possible_masks[0])
            else:
                mask_name = os.path.splitext(img_name)[0] + '.png'
                mask_path = os.path.join(self.masks_dir, mask_name)
        
        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')
        
        image = image.resize(self.img_size, Image.BILINEAR)
        mask = mask.resize(self.img_size, Image.NEAREST)
        
        transform = transforms.ToTensor()
        normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        )
        
        image = transform(image)
        image = normalize(image)
        
        mask = np.array(mask)
        mask = (mask > 128).astype(np.float32)
        mask = torch.from_numpy(mask).unsqueeze(0).float()
        
        return image, mask

In [15]:
def dice_coefficient(pred, target, smooth=1e-6):
    pred_flat = pred.contiguous().view(-1)
    target_flat = target.contiguous().view(-1)
    intersection = (pred_flat * target_flat).sum()
    union = pred_flat.sum() + target_flat.sum()
    return (2. * intersection + smooth) / (union + smooth)

def iou_coefficient(pred, target, smooth=1e-6):
    pred_flat = pred.contiguous().view(-1)
    target_flat = target.contiguous().view(-1)
    intersection = (pred_flat * target_flat).sum()
    total = pred_flat.sum() + target_flat.sum()
    union = total - intersection
    return (intersection + smooth) / (union + smooth)

def save_visualization(images, masks, predictions, epoch, save_dir='visualizations'):
    os.makedirs(save_dir, exist_ok=True)
    
    fig, axes = plt.subplots(min(3, len(images)), 3, figsize=(12, 4*min(3, len(images))))
    if min(3, len(images)) == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(min(3, len(images))):
        img = images[i].cpu().numpy()
        img = img * np.array([0.229, 0.224, 0.225]).reshape(3, 1, 1) + np.array([0.485, 0.456, 0.406]).reshape(3, 1, 1)
        img = np.clip(img, 0, 1)
        img = img.transpose(1, 2, 0)
        
        mask = masks[i][0].cpu().numpy()
        pred = predictions[i][0].cpu().detach().numpy()
        pred_binary = (pred > 0.5).astype(np.float32)
        
        axes[i, 0].imshow(img)
        axes[i, 0].set_title("Image")
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(mask, cmap='gray')
        axes[i, 1].set_title("GT Mask")
        axes[i, 1].axis('off')
        
        axes[i, 2].imshow(pred_binary, cmap='gray')
        axes[i, 2].set_title(f"Prediction")
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig(f'{save_dir}/epoch_{epoch}.png', dpi=100)
    plt.close()

def train():
    config = {
        'batch_size': 4,
        'num_epochs': 50,
        'learning_rate': 0.001,
        'img_size': (256, 256),
        'train_dir': 'dataset/train',
        'val_dir': 'dataset/val',
        'checkpoint_dir': 'checkpoints',
        'device': 'cuda' if torch.cuda.is_available() else 'cpu'
    }
    
    os.makedirs(config['checkpoint_dir'], exist_ok=True)
    
    train_dataset = HumanSegmentationDataset(config['train_dir'], config['img_size'])
    val_dataset = HumanSegmentationDataset(config['val_dir'], config['img_size'])
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config['batch_size'], 
        shuffle=True, 
        num_workers=0
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=config['batch_size'], 
        shuffle=False, 
        num_workers=0
    )
    
    model = HumanSegmentationModel().to(config['device'])
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    
    best_dice = 0
    
    for epoch in range(config['num_epochs']):
        model.train()
        train_loss = 0
        
        for images, masks in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
            images, masks = images.to(config['device']), masks.to(config['device'])
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        avg_train_loss = train_loss / len(train_loader)
        
        model.eval()
        val_dice, val_iou = 0, 0
        val_samples = []
        
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(config['device']), masks.to(config['device'])
                outputs = model(images)
                
                val_dice += dice_coefficient(outputs, masks).item()
                val_iou += iou_coefficient(outputs, masks).item()
                
                if len(val_samples) < 3:
                    val_samples.append((images[:1], masks[:1], outputs[:1]))
        
        avg_val_dice = val_dice / len(val_loader)
        avg_val_iou = val_iou / len(val_loader)
        
        print(f'Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, '
              f'Val Dice: {avg_val_dice:.4f}, Val IoU: {avg_val_iou:.4f}')
        
        if val_samples:
            save_visualization(
                [s[0][0] for s in val_samples],
                [s[1][0] for s in val_samples],
                [s[2][0] for s in val_samples],
                epoch + 1
            )
        
        if avg_val_dice > best_dice:
            best_dice = avg_val_dice
            torch.save({
                'epoch': epoch,
                'model_state': model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
                'val_dice': avg_val_dice,
                'val_iou': avg_val_iou,
            }, f"{config['checkpoint_dir']}/best_model.pt")
            
        torch.save({
            'epoch': epoch,
            'model_state': model.state_dict(),
            'optimizer_state': optimizer.state_dict(),
            'val_dice': avg_val_dice,
            'val_iou': avg_val_iou,
        }, f"{config['checkpoint_dir']}/last_model.pth")
    
    print(f'Best Dice: {best_dice:.4f}')

In [16]:
train()

Epoch 1: 100%|██████████| 600/600 [03:28<00:00,  2.88it/s]


Epoch 1: Train Loss: 0.5639, Val Dice: 0.3585, Val IoU: 0.2207


Epoch 2: 100%|██████████| 600/600 [03:01<00:00,  3.30it/s]


Epoch 2: Train Loss: 0.5311, Val Dice: 0.4369, Val IoU: 0.2817


Epoch 3: 100%|██████████| 600/600 [03:10<00:00,  3.15it/s]


...

Epoch 48: Train Loss: 0.0742, Val Dice: 0.7488, Val IoU: 0.6048


Epoch 49: 100%|██████████| 600/600 [07:06<00:00,  1.41it/s]


Epoch 49: Train Loss: 0.0727, Val Dice: 0.7479, Val IoU: 0.6046


Epoch 50: 100%|██████████| 600/600 [07:12<00:00,  1.39it/s]


Epoch 50: Train Loss: 0.0739, Val Dice: 0.7587, Val IoU: 0.6186
Best Dice: 0.7624
